In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [4]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
train.head()

,filename,0,1,2,3,4,5,6,7,8,...,13,14,15,16,17,18,19,20,21,22
0,8272,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,7833,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,8844,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,6586,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,7666,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [7]:
d = {"filename" : [],
     "text" : []}
path = ".\\data\\train"
cs = os.listdir(path)

for c in cs:
    fcs = path+"\\"+c
    fns = os.listdir(fcs)
    for fn in fns :
        f_path = path+"\\"+c+"\\"+fn

        text_f = open(f_path, "r").read().replace('\n', '')
        d['filename'].append(fn)
        d['text'].append(text_f)

d['filename']=list(map(lambda s: s.lstrip("0"), d["filename"]))
df = pd.DataFrame(d)
df = df.drop(df[df['filename']==''].index)
df['filename'] = df['filename'].astype(np.int64)
df

,filename,text
0,21,Augmentation mentoplasty using Mersilene mesh....
1,33,Multiple intracranial mucoceles associated wit...
2,52,Replacement of an aortic valve cusp after neon...
3,306,The value of indium 111 leukocyte scanning in ...
4,334,Febrile infants less than eight weeks old. Pre...
...,...,...
10428,9991,Afferent nipple valve malfunction caused by an...
10429,9992,Extracorporeal shock wave lithotripsy in combi...
10430,9993,Tamm-Horsfall autoantibodies in interstitial c...
10431,9995,Results of contemporary radical cystectomy for...


In [8]:
train_df = df.merge(train, on="filename", how='inner')
train_df.head(2)

,filename,text,0,1,2,3,4,5,6,7,...,13,14,15,16,17,18,19,20,21,22
0,21,Augmentation mentoplasty using Mersilene mesh....,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,21,Augmentation mentoplasty using Mersilene mesh....,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
# Create and fit the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df.text)
# Create the (X, Y) variables
X_train = tokenizer.texts_to_sequences(train_df.text)
X_train = pad_sequences(X_train, maxlen=400)
#Y_train = to_categorical(train_df.target)

In [10]:
X_train.shape

(10432, 400)

In [11]:
df1 = train_df.drop(['filename', 'text'], axis=1)
Y_train = df1.to_numpy()

In [12]:
Y_train.shape

(10432, 23)

In [17]:
from keras.models import Sequential                                    
from keras.layers import Flatten, Dense 
from keras.layers import Embedding
from keras.layers import LSTM

# Build the model
model = Sequential()
model.add(Embedding(100000, 128))
model.add(LSTM(128, dropout=0.2))
# Output layer has 23 units and uses `softmax`
model.add(Dense(23, activation="softmax"))
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [19]:
# Train the model
model.fit(X_train, Y_train,           
          batch_size=64, 
          epochs=50,
          verbose=2)

Epoch 1/50
163/163 - 176s - loss: 0.0480 - accuracy: 0.5133
Epoch 2/50
163/163 - 174s - loss: 0.0462 - accuracy: 0.5304
Epoch 3/50
163/163 - 172s - loss: 0.0449 - accuracy: 0.5485
Epoch 4/50
163/163 - 168s - loss: 0.0437 - accuracy: 0.5573
Epoch 5/50
163/163 - 167s - loss: 0.0428 - accuracy: 0.5627
Epoch 6/50
163/163 - 168s - loss: 0.0422 - accuracy: 0.5696
Epoch 7/50
163/163 - 173s - loss: 0.0415 - accuracy: 0.5737
Epoch 8/50
163/163 - 170s - loss: 0.0409 - accuracy: 0.5819
Epoch 9/50
163/163 - 167s - loss: 0.0403 - accuracy: 0.5835
Epoch 10/50
163/163 - 163s - loss: 0.0399 - accuracy: 0.5904
Epoch 11/50
163/163 - 163s - loss: 0.0395 - accuracy: 0.5895
Epoch 12/50
163/163 - 161s - loss: 0.0392 - accuracy: 0.5883
Epoch 13/50
163/163 - 161s - loss: 0.0388 - accuracy: 0.5962
Epoch 14/50
163/163 - 162s - loss: 0.0385 - accuracy: 0.5984
Epoch 15/50
163/163 - 164s - loss: 0.0383 - accuracy: 0.5919
Epoch 16/50
163/163 - 160s - loss: 0.0381 - accuracy: 0.5951
Epoch 17/50
163/163 - 162s - loss

In [20]:
d = {"filename" : [],
     "text" : []}
path = ".\\data\\test"
cs = os.listdir(path)

for fn in cs:
    f_path = path+"\\"+fn
    text_f = open(f_path, "r").read().replace('\n', '')
    d['filename'].append(fn)
    d['text'].append(text_f)

In [21]:
d['filename']=list(map(lambda s: s.lstrip("0"), d["filename"]))
test_df = pd.DataFrame(d)
test_df['filename'] = test_df['filename'].astype(np.int64)
test_df

,filename,text
0,1,Cause of death in an emergency department A r...
1,10,Mistaken diagnoses in continuous muscle fiber ...
2,11,Haemophilus influenzae meningitis with prolong...
3,12,Dystrophin analysis in the differential diagno...
4,13,EEG correlation of improvement in hemolytic-ur...
...,...,...
7638,19988,Clinical utility of a position-monitoring cath...
7639,19989,Atracurium decay and the formation of laudanos...
7640,19990,The induction dose of propofol in infants 1-6 ...
7641,19992,Postoperative myocardial ischemia in patients ...


In [22]:
X_test = tokenizer.texts_to_sequences(test_df.text)
X_test = pad_sequences(X_test, maxlen=400)
X_test.shape

(7643, 400)

In [23]:
preds = np.round(model.predict(X_test),0).astype(int)
len(preds)

7643

In [24]:
filename = list(test_df['filename'].values)

In [25]:
submission = pd.DataFrame(preds, columns=['target_00', 'target_01',	'target_02', 'target_03',	'target_04',	
                             'target_05',	'target_06',	'target_07', 'target_08', 'target_09',
                             'target_10','target_11','target_12','target_13','target_14', 'target_15',
                             'target_16', 'target_17', 'target_18', 'target_19', 'target_20',
                             'target_21', 'target_22'])

In [26]:
submission['filename'] = filename

In [27]:
submission.to_csv('sub.csv', index=False)

##### Score : 0.92765